## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import geocode
import os
from shapely.ops import nearest_points

In [2]:
#create a txt file
line0="id;addr\n"
line1="0; Yliopistonkatu 4, 00100 Helsinki, Finland\n"
line2="1;Ullanlinna Helsinki Finland"
# read in input files
f = open(r"C:\Users\USER PC\Documents\HelsinkiUni\exercise-3-LeilaMemoi\activity_locations.txt","w",encoding="utf-8")
f.writelines([line0,line1,line2])
f=open(r"C:\Users\USER PC\Documents\HelsinkiUni\exercise-3-LeilaMemoi\activity_locations.txt","r")
print(f.read())
f.close()

id;addr
0; Yliopistonkatu 4, 00100 Helsinki, Finland
1;Ullanlinna Helsinki Finland


In [3]:
data1=pd.read_csv(r"C:\Users\USER PC\Documents\HelsinkiUni\exercise-3-LeilaMemoi\activity_locations.txt",sep=";")
type(data1)

pandas.core.frame.DataFrame

In [4]:
# Geocode activity locations
geo2= geocode(data1["addr"],provider="nominatim",user_agent="app")
print(geo2)

                    geometry  \
0  POINT (24.94874 60.16972)   
1  POINT (24.94940 60.15871)   

                                             address  
0  UniSport, Kluuvi, 4, Yliopistonkatu, Kaisaniem...  
1  Ullanlinna, Eteläinen suurpiiri, Helsinki, Hel...  


In [5]:
# Check coordinate reference system information (re-project/define if needed)
geo2.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
geo2=geo2.to_crs(epsg=3879)

In [7]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:
data2=pd.read_csv(r"C:\Users\USER PC\Documents\HelsinkiUni\exercise-3-LeilaMemoi\shopping_centers.txt",sep=";")
shopping_centres= geocode(data2["addr"],provider="nominatim",user_agent="app")
shopping_centres =shopping_centres.merge(data2, left_index=True, right_index=True, how='inner')
shopping_centres=shopping_centres[["geometry","address","name"]]
shopping_centres

,geometry,address,name
0,POINT (25.08294 60.21170),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",Itis
1,POINT (24.93828 60.16878),"Salaattiasema, 14-20, Mannerheimintie, Keskust...",Forum
2,POINT (24.73805 60.16022),"Captain Corvus, 11, Piispansilta, Matinkylä, S...",Iso-omena
3,POINT (24.81042 60.21752),"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...",Sello
4,POINT (24.96282 60.29245),"Stockmann, 3, Vantaanportinkatu, Vantaanportti...",Jumbo
5,POINT (24.97904 60.18702),"Silta, 5, Hermannin rantatie, Verkkosaari, Kal...",REDI
6,POINT (24.93026 60.19846),"Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...",Tripla


In [8]:
def get_nearest_values(row, destination, point_column='geometry', value_column="name"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    destination_unary = destination["geometry"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], destination_unary)
    
    # Get corresponding values from the other df
    nearest_data = destination.loc[destination["geometry"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [9]:
geo2["nearest_location"] = geo2.apply(get_nearest_values, destination=shopping_centres, point_column="geometry", axis=1)
geo2

,geometry,address,nearest_location
0,POINT (25497154.366 6672982.716),"UniSport, Kluuvi, 4, Yliopistonkatu, Kaisaniem...",Itis
1,POINT (25497190.277 6671756.866),"Ullanlinna, Eteläinen suurpiiri, Helsinki, Hel...",Itis


In [10]:
print("Shopping center closest to home:", geo2.iloc[-1,-1])
print("Shopping centre closest to work:", geo2.iloc[0,-1])

Shopping center closest to home: Itis
Shopping centre closest to work: Itis


Can you think of other application cases for the nearest neighbour analysis?